In [12]:
import sys, os
from pyspark.sql import SparkSession
from pyspark import SparkConf
from jobs.ncaa_basketball_schema import bb_schema
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

from pyspark.sql.functions import col

# This allows Plotly plots to show in Jupyter
init_notebook_mode(connected=True)
    
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.2.5 pyspark-shell'



conf = SparkConf().setAppName("ncaa_basketball") \
    .setMaster("spark://asprague-laptop:7077") \
    .set("spark.driver.memory", "4g")
spark= SparkSession.builder.config(conf=conf).getOrCreate()

<h1>Writing to PostgreSQL</h1>

<h3>Get the Data</h3>

In [13]:
bb_csv_df = spark.read \
    .csv("./docs/*.csv", schema=bb_schema, header=True)

shooting_events = ['twopointmiss', 'twopointmade','threepointmiss','threepointmade']


In [14]:
URL_POSTGRESQL = "jdbc:postgresql://{host}:{port}/{dbname}"

In [15]:
shooting_events = ['twopointmiss', 'twopointmade','threepointmiss','threepointmade']



bb_csv_df.where(col("event_type").isin(shooting_events)).limit(5).write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("driver", "org.postgresql.Driver") \
    .option("url", URL_POSTGRESQL.format(host="localhost", port=9876, dbname="ncaa_mens_basketball")) \
    .option("dbtable", "ncaa_events") \
    .option("user", "asprague") \
    .option("password", "password") \
    .save()

# options = {
#     'driver': "org.postgresql.Driver",
#     'url': URL_POSTGRESQL.format(host="localhost", port=9876, dbname="ncaa_mens_basketball"),
#     'dbtable': "ncaa_events",
#     'user': "asprague",
#     'password': "password"
# }

# bb_csv_df.write \
#     .format("jdbc") \
#     .options(**options) \
#     .save()



<h1>Reading From PostgreSQL</h1>

In [ ]:
bb_df = spark.read \
    .format("jdbc") \
    .option("driver", "org.postgresql.Driver") \
    .option("url", URL_POSTGRESQL.format(host="localhost", port=9876, dbname="ncaa_mens_basketball")) \
    .option("dbtable", "ncaa_events") \
    .option("user", "asprague") \
    .load()

# bb_df.printSchema()

<h1>Examine Shooting Percentage by Distance From the Basket</h1>

<h3>Available Columns</h3>
<ul>
    <li>Event Type (shooting)</li>
    <li>Event X Coord</li>
    <li>Event Y Coord</li>
    <li>Home Division Alias</li>
    <li>Away Division Alias</li>
</ul>

<h3>Process</h3>
<ul>
    <li>Filter the Shooting Events</li>
    <li>Filter for the D1 games</li>
    <li>Convert the event coord to distance from the basket</li>
    <li>Get the feet from the basket, group by feet</li>
    <li>Sum the makes at that foot and the makes</li>
    <li>Sum the attempts at that foot and the makes</li>
    <li>Apply percentage to that foot</li>
    <li>Plot the results</li>
</ul>

<h5>Filter the Shooting Events</h5>

In [5]:
# What are the events?
bb_csv_df.select(col("event_type")) \
    .groupBy("event_type").count() \
    .orderBy("event_type").show(100, truncate=False)


+---------------------+------+
|event_type           |count |
+---------------------+------+
|assist               |293403|
|attemptblocked       |81863 |
|block                |81863 |
|clearpathfoul        |53    |
|deadball             |55249 |
|defensivethreeseconds|4     |
|delay                |60    |
|ejection             |53    |
|endperiod            |23128 |
|flagrantone          |1778  |
|flagranttwo          |118   |
|freethrow            |2     |
|freethrowmade        |316637|
|freethrowmiss        |134256|
|jumpball             |6157  |
|kickball             |1660  |
|laneviolation        |242   |
|lineupchange         |35926 |
|minortechnicalfoul   |89    |
|offensivefoul        |48713 |
|officialtimeout      |7528  |
|openinbound          |6847  |
|opentip              |11133 |
|personalfoul         |243613|
|possession           |390   |
|rebound              |790490|
|review               |3370  |
|shootingfoul         |183541|
|substitution         |466   |
|teamtim

In [16]:
shooting_events = ['twopointmiss', 'twopointmade','threepointmiss','threepointmade']

shooting_only = bb_csv_df.where(col("event_type").isin(shooting_events))

<h5>Filter for DI Games</h5>

In [17]:
division_one_only = shooting_only \
    .where(col("home_division_alias").isin("D1")) \
    .where(col("away_division_alias").isin("D1"))

In [18]:
# Select the coordinates, basket, and if the shot was made

shots = division_one_only.select(col("event_coord_x"),
                                       col("event_coord_y"),
                                       col("team_basket"),
                                       col("shot_made")).sample(False, 0.01)

In [19]:
shots.count()

12625

In [20]:
x_axis = []
y_axis = []

for shot in shots.collect():
    x_axis.append(shot['event_coord_x'])
    y_axis.append(shot['event_coord_y'])
    
shot_data = [go.Scatter(x=x_axis, y=y_axis, mode="markers")]


shots_fig = go.Figure(data=shot_data)

iplot(shots_fig, config={'displaylogo': False})

In [10]:
# Filter for the Left Basket

left_basket = shots.where(col("team_basket") == "left")

In [11]:
# Find the shot distance
from pyspark.sql.functions import hypot, ceil, when

RIGHT_BASKET_X = 1065
LEFT_BASKET_X = 63
BASKET_Y = 50 * 12 / 2


shot_distance = left_basket \
    .withColumn("x_shot_distance", col("event_coord_x") - LEFT_BASKET_X) \
    .withColumn("y_shot_distance", col("event_coord_y") - BASKET_Y) \
    .withColumn("shot_distance", hypot(col("x_shot_distance"), col("y_shot_distance"))) \
    .select(col("shot_distance"),col("shot_made")) \
    .orderBy("shot_distance") \
    .withColumn("shot_feet", ceil(col("shot_distance") / 12))

In [12]:
# Get the total shots and the made shots and group each df by feet
# The Three-Point arc 

total_shots_by_distance = shot_distance.groupBy("shot_feet").count().orderBy("shot_feet")

shots_made_by_distance = shot_distance.where(col("shot_made") == True).groupBy("shot_feet").count()

In [21]:
# Join the DataFrames on Feet

shot_made = shots_made_by_distance \
    .join(total_shots_by_distance, 
          shots_made_by_distance["shot_feet"] == total_shots_by_distance["shot_feet"]) \
    .orderBy(total_shots_by_distance["shot_feet"])


NameError: name 'shots_made_by_distance' is not defined

In [14]:
shot_percentage_by_feet = shot_made \
    .withColumn("fg_percentage", 
                shots_made_by_distance['count'] / total_shots_by_distance['count'])



<h5>Convert the Event Coordinates to the basket</h5>

In [15]:
x = []
y = []

for distance in shot_percentage_by_feet.collect():
    x.append(distance['shot_feet'])
    y.append(distance['fg_percentage'])

fg_per_by_foot = [go.Bar(x=x, y=y)]

shot_percent_fig = go.Figure(data=fg_per_by_foot)

iplot(shot_percent_fig, config={'displaylogo': False})

In [16]:
with_shot_points = shot_percentage_by_feet.withColumn("points", when(total_shots_by_distance["shot_feet"] < 21, 2).otherwise(3))

expected_value = with_shot_points.withColumn("ev", col("points") * col("fg_percentage"))

x_per = []
y_per = []

for distance in expected_value.collect():
    x_per.append(distance['shot_feet'])
    y_per.append(distance['ev'])

ev_by_foot = [go.Bar(x=x_per, y=y_per)]

ev_by_foot_fig = go.Figure(data=ev_by_foot)
iplot(ev_by_foot_fig, config={'displaylogo': False})


In [23]:
spark.stop()